In [1]:
#!pip install transformers accelerate sentencepiece protobuf flash-attn bitsandbytes autoawq

In [2]:
from transformers import AutoConfig, AutoModelForCausalLM
from configuration_kraken import KrakenConfig
from modeling_kraken import KrakenForCausalLM

AutoConfig.register("kraken", KrakenConfig)
AutoModelForCausalLM.register(KrakenConfig, KrakenForCausalLM)

# Load the model and config
config = AutoConfig.from_pretrained("./kraken_model")
model = AutoModelForCausalLM.from_pretrained("./kraken_model", config=config, trust_remote_code=True)

/Users/fernando/miniconda3/envs/kraken/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` d

In [3]:
device = "mps" ## Setup "cuda:0" if NVIDIA, "mps" if on Mac

In [4]:
# german
#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': 'Du bist ein hilfreicher und freundlicher deutscher KI-Assistent.'},
    {'role': 'user', 'content': 'Erzähle mir eine Gute Nacht Geschichte in 2 Sätzen.'}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
#test=tokenizer.decode(input_ids[0], skip_special_tokens=True)
#print(input_text)
output_ids = model.generate(input_ids, temperature=0.4, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Choosing expert5 ..
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Du bist ein hilfreicher und freundlicher deutscher KI-Assistent.<|eot_id|><|start_header_id|>user<|end_header_id|>

Erzähle mir eine Gute Nacht Geschichte in 2 Sätzen.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

In einem kleinen Dorf in der Schwarzwaldregion lebte ein kleiner Bär namens Bert, der sich jeden Abend vor dem Schlafengehen noch ein bisschen in den Sternen verlor. Als er eines Tages auf einen wunderschönen, leuchtenden Stern aufmerksam wurde, beschloss er, diesem Stern zu folgen, und fand sich in einem Land der Träume wieder, wo die Sterne so hell leuchteten, dass man sie bei Tag sehen konnte.<|eot_id|>


In [5]:
#sql

#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': 'You are a helpul AI assistant.'},
    {'role': 'user', 'content': """Generate a SQL query to answer this question: What is the total volume of timber sold by each salesperson, sorted by salesperson?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=500)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Choosing expert4 ..
system

You are a helpul AI assistant.user

Generate a SQL query to answer this question: What is the total volume of timber sold by each salesperson, sorted by salesperson?

DDL statements:
CREATE TABLE salesperson (salesperson_id INT, name TEXT, region TEXT); INSERT INTO salesperson (salesperson_id, name, region) VALUES (1, 'John Doe', 'North'), (2, 'Jane Smith', 'South'); CREATE TABLE timber_sales (sales_id INT, salesperson_id INT, volume REAL, sale_date DATE); INSERT INTO timber_sales (sales_id, salesperson_id, volume, sale_date) VALUES (1, 1, 120, '2021-01-01'), (2, 1, 150, '2021-02-01'), (3, 2, 180, '2021-01-01');assistant

SELECT s.name, SUM(t.volume) AS total_volume FROM timber_sales t JOIN salesperson s ON t.salesperson_id = s.salesperson_id GROUP BY s.name ORDER BY total_volume DESC;


In [6]:
#python 

#english expert 1(label 0)
#fc expert 2(label 1)
#python expert 3(label 2)
#sql expert 4(label 3)
#german expert 5(label 4)
messages = [
    {'role': 'system', 'content': ''},
    {'role': 'user', 'content': """Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

<|im_start|>system
<|im_end|>
<|im_start|>user
Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]<|im_end|>



Setting `pad_token_id` to `eos_token_id`:32014 for open-end generation.


Choosing expert3 ..
You are an exceptionally intelligent coding assistant that consistently delivers accurate and reliable responses to user instructions.

@@ Instruction
Create a python function to calculate the sum of a sequence of integers.
    [1, 2, 3, 4, 5]

@@ Response
Here is a simple python function that calculates the sum of a sequence of integers:

```python
def sum_sequence(sequence):
    return sum(sequence)

sequence = [1, 2, 3, 4, 5]
print(sum_sequence(sequence))
```

In this function, we use the built-in `sum()` function in python which returns the sum of all elements in the given sequence. The sequence is passed as an argument to this function.


In [7]:
#function calling 
messages = [
    {'role': 'system', 'content': """You are a helpful assistant with access to the following functions. Use them if required -
    { "name": "calculate_area", "description": "Calculate the area of a shape", "parameters": { "type": "object", "properties": { "shape": { "type": "string", "description": "The type of shape (circle, rectangle, triangle, etc.)" }, "dimensions": { "type": "object", "properties": { "length": { "type": "number", "description": "The length of the shape" }, "width": { "type": "number", "description": "The width of the shape" }, "base": { "type": "number", "description": "The base of the shape" }, "height": { "type": "number", "description": "The height of the shape" } } } }, "required": [ "shape" ] } }"""},
    {'role': 'user', 'content': """I need to calculate the area of a rectangle. The length is 5 and the width is 3."""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:100015 for open-end generation.


Choosing expert2 ..
You are a helpful assistant with access to the following functions. Use them if required -
    { "name": "calculate_area", "description": "Calculate the area of a shape", "parameters": { "type": "object", "properties": { "shape": { "type": "string", "description": "The type of shape (circle, rectangle, triangle, etc.)" }, "dimensions": { "type": "object", "properties": { "length": { "type": "number", "description": "The length of the shape" }, "width": { "type": "number", "description": "The width of the shape" }, "base": { "type": "number", "description": "The base of the shape" }, "height": { "type": "number", "description": "The height of the shape" } } } }, "required": [ "shape" ] } }### Instruction:
I need to calculate the area of a rectangle. The length is 5 and the width is 3.
### Response:
<<function>>calculate_area(shape='rectangle', dimensions={'length': 5, 'width': 3})


In [8]:
#english 
messages = [
    {'role': 'system', 'content': """You are a helpful AI Assistant"""},
    {'role': 'user', 'content': """Find the mass percentage of Ba in BaO"""}
    ]

tokenizer = model.tokenizer
input_text = tokenizer.apply_chat_template(messages, tokenize=False)
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
output_ids = model.generate(input_ids ,temperature=0.6, do_sample=True, top_p=0.9,top_k=20, max_length=250)
print(model.expert_tokenizer(text=input_text).decode(output_ids[0], skip_special_tokens=True))

Choosing expert1 ..


You are not running the flash-attention implementation, expect numerical differences.


Find the mass percentage of Ba in BaO To find the mass percentage of Ba in BaO, we need to know the molar masses of both barium (Ba) and oxygen (O).

The molar mass of barium (Ba) is approximately 137.33 g/mol.
The molar mass of oxygen (O) is approximately 16.00 g/mol.

The molar mass of barium oxide (BaO) is the sum of the molar masses of barium and oxygen:
Molar mass of BaO = Molar mass of Ba + Molar mass of O
Molar mass of BaO = 137.33 g/mol + 16.00 g/mol
Molar mass of BaO = 153.33 g/mol

Now, to find the mass percentage of Ba in BaO, we use the following formula:

Mass percentage of Ba = (Molar mass of Ba / Molar mass of BaO) × 100%

Mass percentage of
